In [ ]:
"""
RNN character generator
RNN implementation with Dense layers
There is an RNN layer in pytorch, but in this case we will be using
normal Dense layers to demonstrate the difference between
RNN and Normal feedforward networks.
This is a character level generator, which means it will create character by character
You can input any text file and it will generate characters based on that text
"""
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using '{device}' device")

class RNN(nn.Module):
    """
    Basic RNN block. This represents a single layer of RNN

    """
    def __init__(self, input_size, hidden_size, num_layers, num_classes) -> None:

        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first= True)

        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x) -> tuple[torch.Tensor, torch.Tensor]:
        
        x = self.i2h(x)
        hidden_state = self.h2h(hidden_state)
        hidden_state = torch.tanh(hidden_state + x)
        out = self.h2o(hidden_state)
        return out, hidden_state
    
    def init_zero_hidden(self, batch_size = 1) -> torch.Tensor:
        """ 
        Helper function.
        Returns a hidden state with specified batch size. Defaults to 1
        """

        return torch.zeros(batch_size, self.hidden_size, requires_grad = False)
    


# Custom Dataset Class
class TimeSeriesDataset(Dataset):
    def __init__(self, file_path):
        """
        Reads CSV file and prepares data for batching.
        Assumes the CSV has columns: Time, X, Y, Z.
        """
        # Load data
        data = pd.read_csv(file_path)
        
        # Drop the time column (assume it's not needed for the model)
        self.data = data.iloc[1:, 1:].values  # Extract X, Y, Z
        
        # Create inputs (X) and targets (Y)
        self.X = self.data[:-1]  # All rows except the last
        self.Y = self.data[1:]   # All rows except the first (shifted)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        # Return a single sample (input, target)
        return torch.tensor(self.X[index], dtype=torch.float32), torch.tensor(self.Y[index], dtype=torch.float32)


def create_dataloader(file_path, batch_size, shuffle=False):
    """
    Creates a DataLoader from the given file path.
    """
    dataset = TimeSeriesDataset(file_path)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=True)


# Train Function
def train(model: nn.Module, data: DataLoader, epochs: int, optimizer: optim.Optimizer, loss_fn: nn.Module):
    """
    Trains the model for the specified number of epochs
    Input
    -----
    model: RNN model to train
    data: Iterable DataLoader
    optimizer: Optimizer to use for each epoch
    loss_fn: Function to calculate loss
    """
    train_losses = {}
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train()

    print("=> Starting training")

    for epoch in range(epochs):
        
        epoch_losses = list()
        
        for X, Y in data:  # X, Y are batches
            # Skip last batch if it doesn't match the batch_size
            if X.shape[0] != model.batch_size:
                continue

            # Initialize hidden state
            hidden = model.init_zero_hidden(batch_size=model.batch_size)

            # Send tensors to the device
            X, Y, hidden = X.to(device), Y.to(device), hidden.to(device)

            # Clear gradients
            optimizer.zero_grad()

            # Forward pass
            output, hidden = model(X, hidden)
            loss = loss_fn(output, Y)

            # Backward pass
            loss.backward()

            # Clip gradients to avoid vanishing/exploding gradients
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=3)

            # Update parameters
            optimizer.step()

            # Record batch loss
            epoch_losses.append(loss.item())

        # Store epoch loss
        train_losses[epoch] = torch.tensor(epoch_losses).mean().item()
        print(f"=> Epoch: {epoch + 1}/{epochs}, Loss: {train_losses[epoch]}")

    return train_losses

def save_model(model, file_name):
    torch.save(model.state_dict(), file_name)
    print(f"Model saved to {file_name}")

def load_model(model, file_name):
    model.load_state_dict(torch.load(file_name, weights_only=True))
    model.eval()
    print(f"Model loaded from {file_name}")
    return model

def evaluate(model, dataloader, loss_fn):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    total_samples = 0

    with torch.no_grad():  # No need to track gradients during evaluation
        for X, Y in dataloader:
            X, Y = X.to(device), Y.to(device)

            output, _ = model(X, model.init_zero_hidden(batch_size=X.shape[0]).to(device))

            loss = loss_fn(output, Y)
            total_loss += loss.item() * X.shape[0]  # Weighted by batch size
            total_samples += X.shape[0]

    avg_loss = total_loss / total_samples
    print(f"Test Loss: {avg_loss:.4f}")
    return avg_loss


if __name__ == "__main__":
    # Hyperparameter
    file_path = "D:\Master_EI\FP\Modeling_Dynamic_Systems\DynSys_and_DataSets\lorenz_attractor_dataset.csv"  # Pfad zur CSV-Datei
    batch_size = 32
    input_size = 3  # X, Y, Z (3 Features)
    hidden_size = 256
    output_size = 3  # X, Y, Z (3 Features)
    learning_rate = 0.001
    epochs = 30

    # DataLoader erstellen
    dataloader = create_dataloader(file_path, batch_size)

    # Modell, Optimizer und Verlustfunktion initialisieren
    model = RNN(input_size=input_size, hidden_size=hidden_size, output_size=output_size)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.MSELoss()

    # Modell trainieren
    train_losses = train(model, dataloader, epochs, optimizer, loss_fn)
      
    
    save_model(model, 'rnn_model.pth')

    # Modell nach dem Training laden und evaluieren
    loaded_model = RNN(input_size=input_size, hidden_size=hidden_size, output_size=output_size)
    loaded_model = load_model(loaded_model, 'rnn_model.pth')

    # Testen und Evaluieren (optional, ein Testdatensatz muss vorhanden sein)
    test_file_path = 'D:/Master_EI/FP/Modeling_Dynamic_Systems/DynSys_and_DataSets/lorenz_attractor_dataset_test.csv'  # Beispiel Testdatensatz
    test_dataloader = create_dataloader(test_file_path, batch_size)
    evaluate(loaded_model, test_dataloader, loss_fn)

    print("Training abgeschlossen und Modell evaluiert.")

        


Using 'cpu' device
=> Starting training
=> Epoch: 1/30, Loss: 67.44285583496094
=> Epoch: 2/30, Loss: 20.3575439453125
=> Epoch: 3/30, Loss: 15.087815284729004
=> Epoch: 4/30, Loss: 9.890474319458008
=> Epoch: 5/30, Loss: 5.381691932678223
=> Epoch: 6/30, Loss: 2.555542469024658
=> Epoch: 7/30, Loss: 1.307839035987854
=> Epoch: 8/30, Loss: 0.9152888059616089
=> Epoch: 9/30, Loss: 0.7053211331367493
=> Epoch: 10/30, Loss: 0.6069918870925903
=> Epoch: 11/30, Loss: 0.5540006756782532
=> Epoch: 12/30, Loss: 0.49745607376098633
=> Epoch: 13/30, Loss: 0.5153460502624512
=> Epoch: 14/30, Loss: 0.4463564157485962
=> Epoch: 15/30, Loss: 0.44174879789352417
=> Epoch: 16/30, Loss: 0.3940175473690033
=> Epoch: 17/30, Loss: 0.3542339503765106
=> Epoch: 18/30, Loss: 0.3496111333370209
=> Epoch: 19/30, Loss: 0.35322168469429016
=> Epoch: 20/30, Loss: 0.3436886668205261
=> Epoch: 21/30, Loss: 0.332689493894577
=> Epoch: 22/30, Loss: 0.29688432812690735
=> Epoch: 23/30, Loss: 0.282344251871109
=> Epoch